In [12]:
import sys
sys.path.append("/Users/mayankmaheshwari/Desktop/oodles/")
import uuid
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from scipy.stats import entropy
from oodles import Framework, monitor, Anomaly, DataDriftAlgo, AbstractAnomaly

In [13]:
def generate_data(n):
    """
    This function will generate n rows of sample data.
    params:
        n (Int) : The number of rows you want to generate
    returns:
        A pandas dataframe with n rows.
    """
    data = {
        'uuid' : [str(uuid.uuid4()) for _ in range(n)],
        'feature1' : [random.random() for _ in range(n)],
        'feature2' : [random.random() for _ in range(n)],
        'feature3' : [random.random() for _ in range(n)],
        'target' : [sum([random.random(), random.random(), random.random()]) for _ in range(n)]
    }
    return pd.DataFrame(data)
  
train_df = generate_data(2000)
print(train_df.head())

                                   uuid  feature1  feature2  feature3  \
0  826155b3-216b-441f-83ca-2bd9023d28ca  0.562116  0.056333  0.570388   
1  3d78e8b4-5993-41a5-8cb9-ae8a2fa67160  0.751102  0.659187  0.022919   
2  7bb77e13-4a2c-41fb-98d1-110bcf49d38a  0.302132  0.110781  0.871149   
3  dfe7b48e-21f0-4667-bb10-8039f7a50216  0.479809  0.270178  0.835371   
4  d0c39593-5aa8-48c2-bfb0-1796128dfa7d  0.872977  0.458579  0.554084   

     target  
0  2.038140  
1  1.109876  
2  0.877694  
3  1.478976  
4  1.386923  


In [14]:
# train model
ft_cols = ['feature1', 'feature2', 'feature3']
X = train_df[ft_cols].values
Y = train_df['target'].values

# X_train, X_test_, y_train, y_test = train_test_split(
#     X, Y, test_size=0.3
# )

regressor = XGBRegressor().fit(X, Y)

In [15]:
test_df = generate_data(1500)
test_df.drop(columns = ['target'], inplace = True)

# generate predictions
test_df['prediction'] = test_df[ft_cols].apply(lambda x : regressor.predict([x])[0], axis = 1)

test_df = test_df.rename(columns = {
    'feature1' : 'testfeature1',
    'feature2' : 'testfeature2',
    'feature3' : 'testfeature3'
})

In [17]:

def data_length_normalizer(gt_data, obs_data, bins = 100):
    if len(gt_data) == len(obs_data):
        return gt_data, obs_data 

    # scale bins accordingly to data size
    if (len(gt_data) > 20*bins) and (len(obs_data) > 20*bins):
        bins = 10*bins 

    # convert into frequency based distributions
    gt_hist = plt.hist(gt_data, bins = bins)[0]
    obs_hist = plt.hist(obs_data, bins = bins)[0]
    plt.close()  # prevents plot from showing
    return gt_hist, obs_hist 

def softmax(vec):
    return(np.exp(vec)/np.exp(vec).sum())

def calc_cross_entropy(p, q):
    return entropy(p,q)
    
def calc_drift(gt_data, obs_data, gt_col, obs_col):
    gt_data = gt_data[gt_col].values
    obs_data = obs_data[obs_col].values

    # makes sure the data is same size
    gt_data, obs_data = data_length_normalizer(
        gt_data = gt_data,
        obs_data = obs_data
    )
    # convert to probabilities
    gt_data = softmax(gt_data)
    obs_data = softmax(obs_data)

    # run drift scores
    drift_score = calc_cross_entropy(gt_data, obs_data)
    return drift_score
    
print(calc_drift(gt_data = train_df, obs_data = test_df, gt_col = 'feature1', obs_col = 'testfeature1'))

12.820826125417549
